### Fetching The 7z archive

In [0]:
try:
    dbutils.fs.ls("dbfs:/FileStore/data/dblp.v13.7z")
    print("Archive located. Copying into local store..")
    dbutils.fs.cp("file:/databricks/driver/dblp.v13.7z", "dbfs:/FileStore/data/dblp.v13.7z")
    print("Completed")
except:
    print("7z archive not found. Fetching from URL...")
    !wget https://originalstatic.aminer.cn/misc/dblp.v13.7z
    print("7z archive Downloaded. Moving archive to FileStore..")
    dbutils.fs.mkdirs("dbfs:/FileStore/data")
    dbutils.fs.cp("file:/databricks/driver/dblp.v13.7z", "dbfs:/FileStore/data/dblp.v13.7z")
    print("Completed.")

7z archive not found. Fetching from URL...
--2022-04-14 08:00:47--  https://originalstatic.aminer.cn/misc/dblp.v13.7z
Resolving originalstatic.aminer.cn (originalstatic.aminer.cn)... 159.27.2.14
Connecting to originalstatic.aminer.cn (originalstatic.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2568255035 (2.4G) [application/x-7z-compressed]
Saving to: ‘dblp.v13.7z.2’

dblp.v13.

*** WARNING: max output size exceeded, skipping output. ***

dblp.v13.7z.2       100%[===================>]   2.39G  8.81MB/s    in 4m 46s  

2022-04-14 08:05:34 (8.55 MB/s) - ‘dblp.v13.7z.2’ saved [2568255035/2568255035]

7z archive Downloaded. Moving archive to FileStore..
Completed.


In [0]:
dbutils.fs.ls("file:/databricks/driver/dblp.v13.7z")

Out[31]: [FileInfo(path='file:/databricks/driver/dblp.v13.7z', name='dblp.v13.7z', size=2568255035, modificationTime=1631501339000)]

### Extracting Archive into json

In [0]:
!pip install py7zr

     |████████████████████████████████| 76 kB 3.7 MB/s 
     |████████████████████████████████| 2.4 MB 5.3 MB/s 
     |████████████████████████████████| 132 kB 49.7 MB/s 
     |████████████████████████████████| 58 kB 8.4 MB/s 
     |████████████████████████████████| 2.0 MB 29.3 MB/s 
     |████████████████████████████████| 44 kB 3.3 MB/s 
     |████████████████████████████████| 357 kB 45.9 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
import py7zr

archive = py7zr.SevenZipFile('dblp.v13.7z', mode='r')
archive.extractall()
archive.close()

In [0]:
dbutils.fs.ls("file:/databricks/driver/dblpv13.json")

Out[34]: [FileInfo(path='file:/databricks/driver/dblpv13.json', name='dblpv13.json', size=17352640799, modificationTime=1631240655155)]

### Cleaning NumberInt(#) tags

In [0]:
import re

# Cleaning the `NumberInt` tag
fin = open(f"dblpv13.json")
fout = open(f"dblpv13_clean.json", "wt")
for line in fin:
    fout.write(re.sub(r"NumberInt\([\d]*\)", lambda x: "".join(re.findall(r"\d", x.group(0))), line))
fin.close()
fout.close()

### Partitioning Dataset into JSON files

In [0]:
!pip install ijson tqdm

     |████████████████████████████████| 133 kB 5.2 MB/s 
     |████████████████████████████████| 78 kB 10.1 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
%mkdir data

mkdir: cannot create directory ‘data’: File exists


In [0]:
import ijson
import json
from tqdm import tqdm
import decimal

class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)

data_dir = 'data/'
with open('dblpv13_clean.json', 'r') as f:
    counter, file_id = 0, 0
    file_buffer = []
    for obj_data in tqdm(ijson.items(f, 'item')):
        file_buffer.append(obj_data)
        counter += 1
        if counter % 300000 == 0:
            print(f" Saving, data_PART_{file_id}.json in {data_dir}")
            f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
            dump = json.dumps(file_buffer, cls=DecimalEncoder)
            f.write(dump)
            f.close()
            file_id += 1
            file_buffer = []
f = open(f'{data_dir}data_PART_{file_id}.json', 'w')
dump = json.dumps(file_buffer, cls=DecimalEncoder)
print(f" Saving, data_PART_{file_id}.json in {data_dir}")
f.write(dump)
f.close()
file_id += 1
file_buffer = []

299400it [00:20, 12505.39it/s] Saving, data_PART_0.json in data/
598593it [00:55, 19784.75it/s] Saving, data_PART_1.json in data/
899174it [01:29, 17926.00it/s] Saving, data_PART_2.json in data/
1199741it [02:03, 18730.69it/s] Saving, data_PART_3.json in data/
1499338it [02:37, 18628.16it/s] Saving, data_PART_4.json in data/
16

*** WARNING: max output size exceeded, skipping output. ***

3898449it [07:10, 18690.85it/s] Saving, data_PART_12.json in data/
4198956it [07:45, 15593.51it/s] Saving, data_PART_13.json in data/
4498616it [08:22, 16599.55it/s] Saving, data_PART_14.json in data/
4799667it [08:59, 17460.19it/s] Saving, data_PART_15.json in data/
5098207it [09:36, 29443.27it/s] Saving, data_PART_16.json in data/
5354309it [10:08, 8796.60it/s]
 Saving, data_PART_17.json in data/


In [0]:
len(file_buffer)

Out[40]: 0

In [0]:
# https://stackoverflow.com/a/42540517 (This seems more proper way to merge the frames..)

sc = spark.sparkContext

path = "data/"

dataframes = []
path = "file:/databricks/driver/data/"
for fileinfo in dbutils.fs.ls(path):
    print(f"Reading file {fileinfo[1]} into spark")
    dataframes.append(spark.read.json(fileinfo[0]))
print(len(dataframes))

Reading file data_PART_12.json into spark
Reading file data_PART_9.json into spark
Reading file data_PART_1.json into spark
Reading file data_PART_15.json into spark
Reading file data_PART_7.json into spark
Reading file data_PART_2.json into spark
Reading file data_PART_0.json into spark
Reading file data_PART_13.json into spark
Reading file data_PART_6.json into spark
Reading file data_PART_5.json into spark
Reading file data_PART_16.json into spark
Reading file data_PART_8.json into spark
Reading file data_PART_4.json into spark
Reading file data_PART_3.json into spark
Reading file data_PART_17.json into spark
Reading file data_PART_14.json into spark
Reading file data_PART_11.json into spark
Reading file data_PART_10.json into spark
18


In [0]:
dataframes[0].printSchema()

root
 |-- _id: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- position: string (nullable = true)
 |    |    |-- sid: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- fos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isbn: string (nullable = true)
 |-- issn: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- keywords: array (nullable = true)
 |  

In [0]:
# dbutils.fs.mkdirs("dbfs:/FileStore/data/split_data")

# dbutils.fs.cp("file:/databricks/driver/data/", "dbfs:/FileStore/data/split_data", recurse = True)